In [68]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('words')
from nltk.stem import WordNetLemmatizer #lemitization
from nltk.corpus import words
from tqdm import tqdm
tqdm.pandas()
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

from math import log2

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aless\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aless\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\aless\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [69]:
data = pd.read_csv('../../data/processed/processed_xy_augmented.csv')
change_in_pop = data['test_mean_clicks'].div(data['test_mean_impressions'])-data['clicks'].div(data['impressions'])


In [70]:
def mean_emotion(data,emotion,length):
    means = data.groupby('test_id').mean()[emotion]
    lengths = data[length].apply(lambda x: log2(x+1))
    return data['test_id'].map(lambda x: means.loc[x]).div(lengths)


In [71]:
def emotional_score(data,emotion,length):
    lengths = data[length].apply(lambda x: log2(x+1))
    return data[emotion].div(lengths)


In [72]:
#print(emotional_score(data,'Negative','token_word_count'))

In [73]:
def vizualize_scatter(data,emotion,length,y):
    escore= emotional_score(data,emotion,length) - mean_emotion(data,emotion,length)
    model = LinearRegression().fit(escore.values.reshape(-1,1),y.values.reshape(-1,1))
    #get the gradient of model
    gradient = model.coef_[0][0]
    print(gradient)
    sns.regplot(x=escore,y=y)
    plt.savefig(f'../../data/visualizations//scatter/{emotion}-{length}-scatter.png')
    plt.close()
    

In [74]:
def vizualize_hist(data,emotion,length,y):
    escore= emotional_score(data,emotion,length)
    df = pd.concat([escore,y],axis=1)
    print(df)
    sns.histplot(data=df)
    plt.savefig(f'../../data/visualizations/hist/{emotion}-{length}.png')
    plt.show()
    plt.close()
    

In [75]:
for emotion in ['Positive','Negative','Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust']:
    for length in ['token_word_count','emotive_word_count']:
        vizualize_scatter(data,emotion,length,change_in_pop)